In [11]:
import duckdb
import pandas as pd

##TODO: Falta bajar el dataset directamente de Kaggle
# import kagglehub
# from kagglehub import KaggleDatasetAdapter

# # Load the dataset from KaggleHub
# df = kagglehub.load_dataset(
#     KaggleDatasetAdapter.PANDAS,
#     "alejandroczernikier/properati-argentina-dataset",
#     file_path="/data/raw"
# )




con = duckdb.connect("data/DB/entrenamiento.duckdb")

# Usamos un SELECT sobre read_csv_auto()
con.execute("""
COPY (
    SELECT * FROM read_csv_auto('data/RAW/entrenamiento.csv')
) TO 'data/parquet/entrenamiento.parquet' (FORMAT PARQUET)
""")


# Crear tabla desde el archivo Parquet
# con.execute("""
#     CREATE TABLE entrenamiento AS
#     SELECT * FROM 'data/parquet/entrenamiento.parquet'
# """)



FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [12]:
#print(con.execute("PRAGMA table_info(entrenamiento)").fetchdf())
preview_df = con.execute("SELECT * FROM 'data/parquet/entrenamiento.parquet' LIMIT 10").fetchdf()
print(preview_df)


       id    ad_type start_date   end_date created_on        lat        lon  \
0  556713  Propiedad 2019-11-29 9999-12-31 2019-11-29 -58.442399 -34.573623   
1  192912  Propiedad 2020-06-05 2020-06-08 2020-06-05 -58.430493 -34.606620   
2  238224  Propiedad 2020-07-01 9999-12-31 2020-07-01 -58.491760 -34.574123   
3  257134  Propiedad 2019-08-17 9999-12-31 2019-08-17 -58.420737 -34.631770   
4  521738  Propiedad 2019-08-05 2019-08-31 2019-08-05 -58.429983 -34.607225   
5  383947  Propiedad 2019-10-23 2019-11-08 2019-10-23 -58.420780 -34.608640   
6  986212  Propiedad 2019-08-01 2019-08-14 2019-08-01 -58.371394 -34.621835   
7  499733  Propiedad 2019-07-24 9999-12-31 2019-07-24 -58.404054 -34.599623   
8   76927  Propiedad 2020-06-17 9999-12-31 2020-06-17 -58.507339 -34.590882   
9  473866  Propiedad 2019-07-06 2019-07-09 2019-07-06 -58.421650 -34.588602   

          l1               l2             l3  ... bathrooms surface_total  \
0  Argentina  Capital Federal     Colegiales  ...    

In [13]:
con.execute("DESCRIBE SELECT * FROM 'data/parquet/entrenamiento.parquet'").fetchdf()

,column_name,column_type,null,key,default,extra
0,id,BIGINT,YES,None,None,None
1,ad_type,VARCHAR,YES,None,None,None
2,start_date,DATE,YES,None,None,None
3,end_date,DATE,YES,None,None,None
4,created_on,DATE,YES,None,None,None
5,lat,DOUBLE,YES,None,None,None
6,lon,DOUBLE,YES,None,None,None
7,l1,VARCHAR,YES,None,None,None
8,l2,VARCHAR,YES,None,None,None
9,l3,VARCHAR,YES,None,None,None
